In [ ]:
# Setup and Imports
import requests
import numpy as np
import pandas as pd
import yaml
import datetime 
import base64
import json
import pytz
from typing import Dict, List, Optional, Any
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# from functions import GetToken

print("Libraries imported successfully.")

In [ ]:
def GetToken(username: str, password: str) -> str:
    """Passes through MIDAS username and password and returns the JWT token."""
    credentials = f"{username}:{password}"
    credentials_encodedBytes = base64.b64encode(credentials.encode("utf-8"))
    
    headers = {'Authorization': b'BASIC ' + credentials_encodedBytes}
    url = 'https://midasapi.energy.ca.gov/api/token'

    response = requests.get(url, headers=headers)
    response.raise_for_status() # Raise HTTPError for bad responses
    
    # Assuming token is in the headers, as per your original script
    return response.headers['Token']


In [ ]:
def GetValue(token: str, rateID: str, queryType: str) -> Dict[str, Any]:
    """Retrieves time series data for a specific rateID."""
    headers = {'accept': 'application/json', 'Authorization': "Bearer " + token}
    url = f'https://midasapi.energy.ca.gov/api/valuedata?id={rateID}&querytype={queryType}'
    pricing_response = requests.get(url, headers=headers)
    pricing_response.raise_for_status()
    
    return json.loads(pricing_response.text)

In [ ]:
# Load Credentials and RIN Dictionary
try:
    with open("midas_config.yaml") as fp:
        config = yaml.safe_load(fp)
    midas_config = config.get("midas", {})
    midas_username = midas_config.get("username")
    midas_password = midas_config.get("password")
    print("MIDAS credentials loaded from midas_config.yaml.")
except FileNotFoundError:
    print("ERROR: midas_config.yaml not found. Please create it.")
    midas_username = None
    midas_password = None

In [ ]:
# Define the RIN dictionary (from your repository)
cfh_midas_rin_dict = {
    'SummerHDP':'USCA-CFCF-SHDP-0000',
    'SummerGHG': 'USCA-CFCF-SUDG-0000',
    'SpringHDP': 'USCA-CFCF-SPDP-0000', 
    'SpringGHG': 'USCA-CFCF-SPDG-0000', 
    'WinterHDP': 'USCA-CFCF-WIDP-0000', 
    'WinterGHG': 'USCA-CFCF-WIDG-0000', 
    'FallHDP': 'USCA-CFCF-FLDP-0000', 
    'FallGHG': 'USCA-CFCF-FLDG-0000',
}

print(f"Loaded {len(cfh_midas_rin_dict)} RINs.")

In [ ]:
# def GetToken(username: str, password: str) -> str:
#     """Passes through MIDAS username and password and returns the JWT token."""
#     credentials = f"{username}:{password}"
#     credentials_encodedBytes = base64.b64encode(credentials.encode("utf-8"))
    
#     headers = {'Authorization': b'BASIC ' + credentials_encodedBytes}
#     url = 'https://midasapi.energy.ca.gov/api/token'

#     response = requests.get(url, headers=headers)
#     response.raise_for_status() # Raise HTTPError for bad responses
    
#     # Assuming token is in the headers, as per your original script
#     return response.headers['Token']

In [ ]:
# Define Core API and MidasDataTool Class

# --- Core API Functions (adapted for standalone use) ---





# --- MidasDataTool Class ---

class MidasDataTool:
    """
    A tool to manage authentication, fetch, filter, and export MIDAS rate data.
    """
    def __init__(self, username: str, password: str):
        self.username = username
        self.password = password
        self.utc = pytz.timezone("UTC")
        self.pacific = pytz.timezone("US/Pacific")
        self.token = self._authenticate()


    def _authenticate(self) -> str:
        """Authenticates and retrieves the JWT token."""
        print("Attempting MIDAS authentication...")
        try:
            token = GetToken(self.username, self.password)
            print("Authentication successful.")
            return token
        except Exception as e:
            print(f"Authentication failed. Check credentials and network: {e}")
            raise

    def get_rate_dataframe(self, rin_id: str) -> pd.DataFrame:
        """Fetches 'alldata' for a single RIN ID and converts it to a clean DataFrame."""
        try:
            # Token is used automatically from the class instance
            op = GetValue(self.token, rin_id, "alldata")
        except requests.exceptions.HTTPError as e:
            print(f"Error fetching data for RIN {rin_id}: {e}")
            return pd.DataFrame()

        st_list, et_list, price_list = [], [], []
        
        for value_dict in op.get('ValueInformation', []):
            try:
                # Convert UTC time strings to US/Pacific timezone
                dt_format = "%Y-%m-%d %H:%M:%S"
                st_utc = self.utc.localize(datetime.datetime.strptime(f"{value_dict['DateStart']} {value_dict['TimeStart']}", dt_format))
                et_utc = self.utc.localize(datetime.datetime.strptime(f"{value_dict['DateEnd']} {value_dict['TimeEnd']}", dt_format))

                st_pacific = st_utc.astimezone(self.pacific)
                et_pacific = et_utc.astimezone(self.pacific)
                
                price = value_dict['value']
                
                st_list.append(st_pacific)
                et_list.append(et_pacific)
                price_list.append(price)
            except ValueError as e:
                # Skips rows with unparseable data
                print(f"Data parsing error for RIN {rin_id}: {e}")
                continue

        df = pd.DataFrame({
            'start_time': st_list, 
            'end_time': et_list, 
            'price': price_list,
            'rin_id': rin_id
        }).set_index('start_time').sort_index()
        
        return df

    def filter_and_export(
        self, 
        rin_dict: Dict[str, str],
        rin_filter: Optional[List[str]] = None,
        is_ghg: Optional[bool] = None,
        start_date: Optional[str] = None, 
        end_date: Optional[str] = None,
        export_filename: str = "midas_filtered_data.csv"
    ) -> pd.DataFrame:
        """
        Filters the RIN dictionary, fetches data for matching RINs, applies date 
        filtering, and exports the combined data.
        """
        
        # 1. Apply RIN Filters (rin_filter, is_ghg)
        filtered_rins = {}
        for rin_name, rin_id in rin_dict.items():
            
            if rin_filter and rin_name not in rin_filter:
                continue
                
            if is_ghg is not None:
                is_current_ghg = 'GHG' in rin_name
                if is_ghg != is_current_ghg:
                    continue
            
            filtered_rins[rin_name] = rin_id
            
        if not filtered_rins:
            print("No RINs matched the filtering criteria.")
            return pd.DataFrame()
            
        print(f"RINs selected for fetching: {', '.join(filtered_rins.keys())}")

        # 2. Fetch Data
        all_data_frames = []
        for rin_name, rin_id in filtered_rins.items():
            print(f"Fetching data for {rin_name} ({rin_id})...")
            df = self.get_rate_dataframe(rin_id)
            if not df.empty:
                df['rate_name'] = rin_name
                all_data_frames.append(df)
        
        if not all_data_frames:
            print("No data retrieved for the selected RINs.")
            return pd.DataFrame()
            
        combined_df = pd.concat(all_data_frames)
        
        # 3. Apply Date Filtering (start_date, end_date)
        if start_date:
            try:
                start_ts = pd.to_datetime(start_date, utc=False).tz_localize(self.pacific)
                combined_df = combined_df[combined_df.index >= start_ts]
                print(f"Data filtered from {start_date}")
            except Exception:
                print(f"Warning: Invalid start_date format ({start_date}). Date filtering skipped.")

        if end_date:
            try:
                end_ts = pd.to_datetime(end_date, utc=False).tz_localize(self.pacific)
                # Filter up to, but not including, the end_ts
                combined_df = combined_df[combined_df.index < end_ts] 
                print(f"Data filtered up to {end_date}")
            except Exception:
                print(f"Warning: Invalid end_date format ({end_date}). Date filtering skipped.")
                
        # 4. Export
        combined_df.to_csv(export_filename)
        print(f"\n--- Export successful! ---")
        print(f"Data for {len(filtered_rins)} RINs exported to: {export_filename}")
        print(f"Total rows exported: {len(combined_df)}")
        
        return combined_df

In [ ]:
# Initialize the Tool

if midas_username and midas_password:
    # Initialize the tool; this will authenticate and retrieve the token
    tool = MidasDataTool(midas_username, midas_password)
else:
    print("Cannot initialize tool: Credentials were not loaded successfully.")
    tool = None

In [ ]:
# Example 1 - Filter for ALL Non-GHG rates (HDP) for a specific quarter

if tool:
    df_hdp_quarter = tool.filter_and_export(
        rin_dict=cfh_midas_rin_dict,
        is_ghg=False,  # Filters out rates containing 'GHG' in the name
        start_date='2024-07-01',
        end_date='2024-10-01',
        export_filename='non_ghg_Q3_2024.csv'
    )
    # Display the first few rows of the resulting DataFrame
    print("\nResulting DataFrame (HDP Q3):")
    print(df_hdp_quarter.head())

In [ ]:
# Example 2 - Filter for ONLY 'WinterGHG' and 'SummerGHG' for a specific month

if tool:
    df_ghg_selected = tool.filter_and_export(
        rin_dict=cfh_midas_rin_dict,
        rin_filter=['WinterGHG', 'SummerGHG'], # Filters to only these two specific rates
        start_date='2024-03-01',
        end_date='2024-04-01',
        export_filename='winter_summer_ghg_march.csv'
    )
    print("\nResulting DataFrame (Selected GHG Month):")
    print(df_ghg_selected.head())

In [ ]:
# Initialize the Dash app
app = dash.Dash(__name__)

# Assume 'combined_df' is the result of your MidasDataTool.filter_and_export()
# We use this to populate the filter options
rate_options = [{'label': i, 'value': i} for i in combined_df['rate_name'].unique()]

app.layout = html.Div([
    html.H1("Dynamic Rate and Timescale Plotter"),

    # --- Rate Filter (Dynamic Dropdown) ---
    dcc.Dropdown(
        id='rate-dropdown',
        options=rate_options,
        value=rate_options[0]['value'] if rate_options else None, # Default to the first rate
        multi=True,
        placeholder="Select Rate(s)",
        style={'width': '50%'}
    ),
    
    # --- Timescale Filter (Date Range) ---
    dcc.DatePickerRange(
        id='date-range-picker',
        start_date=combined_df.index.min().date(),
        end_date=combined_df.index.max().date(),
        display_format='YYYY-MM-DD',
    ),

    # --- Plot Output ---
    dcc.Graph(id='price-time-series-plot')
])